# Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models

This notebook shows an example for the threat model and attack described in "Decepticons: Corrupted Transformers Breach Privacy in Federated Learning for Language Models
". This example deviates from the other "honest-but-curious" server models and investigates a malicious server that may send malicious server updates. The attack succeeds for a range of common transformer architectures and works merely by sending a single malicious query to the user model.

In this notebook, we attack the tiny transformer model discussed in "Advances and Open Problems in Federated Learning" (https://arxiv.org/abs/1912.04977). The model architecture is unchanged.



Paper URL: https://arxiv.org/abs/2201.12675

### Abstract:
A central tenet of Federated learning (FL), which trains models without centralizing user data, is privacy. However, previous work has shown that the gradient updates used in FL can leak user information. While the most industrial uses of FL are for text applications (e.g. keystroke prediction), nearly all attacks on FL privacy have focused on simple image classifiers. We propose a novel attack that reveals private user text by deploying malicious parameter vectors, and which succeeds even with mini-batches, multiple users, and long sequences. Unlike previous attacks on FL, the attack exploits characteristics of both the Transformer architecture and the token embedding, separately extracting tokens and positional embeddings to retrieve high-fidelity text. This work suggests that FL on text, which has historically been resistant to privacy attacks, is far more vulnerable than previously thought.

### Startup

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("..")
    import breaching
    
import torch
%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [ ]:
cfg = breaching.get_config(overrides=["case=9_bert_training", "case/data=cola", "case.data.task=classification",
                                      "attack=decepticon", "case/server=malicious-transformer"])    
    
device = torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [ ]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 0 # From which user?
cfg.case.data.shape = [8] # This is the sequence length

cfg.case.server.pretrained = True

cfg.case.server.has_external_data = False  # Not strictly necessary, could also use random text (see Appendix)
cfg.case.data.tokenizer = "word-level"
cfg.attack.token_strategy = "embedding-norm"

cfg.attack.embedding_token_weight=0

cfg.attack.matcher = "abs-corrcoef"
# Attack hyperparameters:

# this option requires installation of `k-means-constrained` which can be tricky:
# If this doesn't work for you, falling back to "dynamic-threshold" is still a decent option.
cfg.attack.sentence_algorithm = "k-means" 

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [ ]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [ ]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)  

In [ ]:
shared_data["gradients"][0].norm(dim=1).topk(k=5)

In [ ]:
user.print(true_user_data)

### Reconstruct user data:

Now we launch the attack, reconstructing user data based on only the `server_payload` and the `shared_data`. 

For this attack, we also share secret information from the malicious server with the attack (`server.secrets`), which here is the location and structure of the imprint block.

In [ ]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], server.secrets, 
                                                      dryrun=cfg.dryrun)

Next we'll evaluate metrics, comparing the `reconstructed_user_data` to the `true_user_data`.

In [ ]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, order_batch=True, compute_full_iip=False, 
                                    cfg_case=cfg.case, setup=setup)
min(metrics["intra-sentence_token_acc"]), max(metrics["intra-sentence_token_acc"])

And finally, we also plot the reconstructed data:

In [ ]:
user.print(reconstructed_user_data)

In [ ]:
user.print_with_confidence(reconstructed_user_data)

In [ ]:
user.print_and_mark_correct(reconstructed_user_data, true_user_data)

### Notes:
* There are a variety of hyperparameters to the attack which are set to reasonable defaults. Performance of the attack could be improved in some unusual use cases (datasets or models) by tuning these parameters further.
* In this example, dropout is disabled under the assumption that this is a parameter that can be controlled in the server update. The optimal attack simply disables dropout. However, the attack can still succeed when dropout is enforced by the user, albeit with a minor loss in reconstruction quality.
* Try increasing `num_data_points` or `data.shape`